**Configure Pub/Sub notifications for Cloud Storage**
- reference link: https://cloud.google.com/functions/docs/tutorials/terraform-pubsub
- config so that change in your bucket will send notification to pubsub

**Enable services and aips for this reading:**
- list available services and aips. Command: gcloud services list --available
- list enabled servcies and aips. Command: gcloud services list --enabled
- enable services or api. Command: gcloud services enable $service_name

**Input variable**
- input variable help you to store values into variable so that we can build terraform which can apply different config with the same template
- we declare variable with keyword variable
- some basic arguments for variable: default, type, description, validaiton, sensitive, nullable.
- using variable, to access variable value we use var.variablename
- assign value 

**Setting up your environment**
- enable services and apis need to this reading (cloud storage, cloud function, cloud build, artifact registry)
- enable services and apis with command: gcloud services enable $service_name
- install gcloud cli
- update gcloud cli with command: gcloud components update
- set up environment for node js development

**Note about terraform resource in gcp**
- reference link about terraform provider document: https://registry.terraform.io/providers/hashicorp/google/latest/docs. We can reference this docs for all resources in gcp
- google_pubsub_topic which will create a pubsub topic. 
- google_storage_bucket to create a bucket in cloud storage. Once bucket is created it's location can not be change.
- note about location in cloud storage. we can set bucket locationi with region (stored in one region data center), duo-region (stored in two region data center), multi-region (stored data in multiple data center region). Reference about location in cloud storage: https://cloud.google.com/storage/docs/locations
- google_storage_bucket_object: to upload or create an object in given bucket.
- google_cloudfunctions2_function: to create a cloud function in google cloud


**Note about terraform cloud function**
- reference about cloud function in terraform: https://registry.terraform.io/providers/hashicorp/google/latest/docs/resources/cloudfunctions2_function
- immport arguments:
    - name (required): name of the cloud function
    - build_config: build steps to build container from source code.
    - service_config: server config to run the code
    - event_trigger: event config which trigger cloud function
    - entry_point: the name of the function in source code
- import config list: name, build_config, service_config, event_trigger.
- command to list runtimes: gcloud functions runtimes list --region=$region_name | grep python

**Configure Pub/Sub notifications for Cloud Storage**
- goal: given we have cloud storage bucket and pubsub topic. We want to config so that when we have change in cloud storage it will public a notification on pubsub topic
- reference link: https://cloud.google.com/storage/docs/reporting-changes
- apply notification configuration: 
- get a notification configuration of a given notification: gcloud storage buckets notifications describe projects/_/buckets/BUCKET_NAME/notificationConfigs/NOTIFICATION_ID
- list notification configurations for a bucket: gcloud storage buckets notifications list gs://BUCKET_NAME
- remove a notification configuration: gcloud storage buckets notifications delete projects/_/buckets/BUCKET_NAME/notificationConfigs/NOTIFICATION_ID
- we can apply or remove notification configuration using:
    - gcloud cli
    - terraform


In [ ]:
#to list predefined roles of a service with gcloud command
gcloud iam roles list | grep $service_name
gcloud iam roles list | grep pubsub


In [ ]:
#add pubsub admin role for terraform service account
#to check the role of a service account
gcloud projects add-iam-policy-binding $project_id --member="serviceAccount:$terraform_service_account" --role="$role_four"

In [ ]:
#binding member with given role to a service account (resource). So that member (service account) can act as a service account
gcloud iam service-accounts add-iam-policy-binding $terraform_service_account --member="serviceAccount:$terraform_service_account" --role="roles/iam.serviceaccounts.actAs"
gcloud iam service-accounts add-iam-policy-binding 149838564778-compute@developer.gserviceaccount.com --member="serviceAccount:$terraform_service_account" --role="roles/iam.serviceAccountUser"

**Note about service account problem for google_pubsub_topic_iam_binding, google_cloudfunctions2_function**
- here we do not create new service account but use terraform service account
- google_pubsub_topic_iam_binding:
    - pubsub topic is a resource
    - google_pubsub_topic_iam_binding: to binding service_account which specific role to a given topic resource
- google_cloudfunctions2_function:
    - because we use custom terraform_service_account, therefore we need to assign the server to run as custom service account
    - we binding member with given role (oles/iam.serviceaccounts.actAs) to resource (service account), so that member and act as service account

In [ ]:
#check bucket created
gcloud storage ls
#check objects in bucket
gcloud storage ls $bucket_path
#copy files from local machine to cloud storage bucket
gcloud storage cp "./src/test.txt" "gs://2bb951bd4a5af590-gcf-source/"
gcloud storage cp "./src/test.json" "gs://2bb951bd4a5af590-gcf-source/"

In [ ]:
#check pubsub resource
gcloud pubsub topics list


In [ ]:
#check cloud function will trigger given file upload
gcloud functions logs read function
